In [4]:
import tempfile
import formulallm.formula as f
from autogen import ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor
from autogen import register_function
from typing import Annotated
import io
import contextlib

In [5]:
local_llm_config={
    "config_list": [
        {
            "model": "NotRequired", # Loaded with LiteLLM command
            "api_key": "NotRequired", # Not needed
            "base_url": "http://0.0.0.0:4000"  # Your LiteLLM URL
        }
    ],
    "cache_seed": None # Turns off caching, useful for testing different models
}

In [6]:
dsl_path = "./data/MappingExample.4ml"
domain = "Mapping"
partial_model = "pm"

In [19]:
code_fixer = ConversableAgent(
    name = "Code Fixer",
    llm_config = local_llm_config,
    human_input_mode="NEVER",
    system_message = '''You are a code fixer.
    You will be given a DSL domain-and-paritial-model pair written in a DSL called formula.
    The partial model is unsolvable under the domain constraints, and you will be given some least unsatisfiable core condition
    as a hint to suggest repairs to the DSL code. Your goal is to fix the partial model to make it solvable.
    When suggesting repairs, prioritize modifying the partial model over modifying the domain constraints unless otherwise specified. 

    Note: to fix a partial model does not mean to solve the variables in the model.
    
    WorkFlow:
    1. Understand the domain and the constraints.
    2. Using the constraints from the domain, modify the partial model. 
    3. Reply with MODIFIED CODE ONLY. 
    Don't say anything that is not Formula code, because you will need to save your output to the original file!!
    4. Save the whole chunck of modified DSL code by calling the rewrite_dsl_file(). The input is the output you generated!

    Important note: 
    1. Please return the complete code, i.e. the modified domain-model pair, because you need to save the code to the file. 
    The user will not revise your output!
    2. Do not call any function other than rewrite_dsl_file()!!!!!
'''
)

In [20]:
user_proxy = ConversableAgent(
    name = "User",
    llm_config = local_llm_config,
    human_input_mode="ALWAYS",  # Always take human input for this agent for safety.
    system_message="You are the user proxy."
    "Please obey the following workflow strictly!"
    "When you receive a message from the fixer,"
    "call the test_solve() function to reload, solve, and extract the solution of the modified DSL."
    "Then ask the human in the loop to provide any feedback"
)

In [21]:
made_test_solve = False
task_id = 1

In [22]:
def test_solve() -> Annotated[str, "The content written in DSL of the file loaded"]:

    global made_test_solve
    made_test_solve = True
    
    global dsl_path, domain, partial_model, tasd_id
    code = f.load(dsl_path)
    f.solve(partial_model, "1", f"{domain}.conforms")
    f.extract(str(task_id), '0', '0')
    task_id += 1
    return code 

In [23]:
def rewrite_dsl_file(generated_dsl_code: Annotated[str, "The DSL code generated by the fixer agent"]) -> Annotated[
    None, "This function rewrites the original file with the DSL code generated by the fixer agent."]:
    
    global made_test_solve
    made_test_solve = True

    global dsl_path
    try:
        # Open the file specified by file_path in write mode ('w')
        with open(dsl_path, 'w') as file:
            # Write the generated DSL code to the file
            file.write(generated_dsl_code)
        # Print a success message indicating the file has been rewritten
        print(f"File {dsl_path} successfully rewritten with the generated DSL code.")
    except Exception as e:
        # Print an error message if an exception occurs during the file operation
        print(f"An error occurred while rewriting the file: {e}")
    

In [24]:
def terminate_nested_chat(msg):
    global made_test_solve
    if made_test_solve:
        made_test_solve = False
        return True
    else:
        return False

In [25]:
executor = ConversableAgent(
    name = "Executor",
    llm_config=False,
    human_input_mode="NEVER",
    is_termination_msg=terminate_nested_chat,
    default_auto_reply="Please continue."
)

In [26]:
register_function(
        test_solve,
        caller = user_proxy,
        executor=executor,
        name="test_solve",
        description='''This function does three tasks:
        1. Check the syntax of the DSL by trying to load the file
        2. Solve the partial model
        3. Get the result of the last "solve" task. 
        If the partial model is solvable,
        will output the solution to the model; 
        If not, will output the least unsatisfied core condition as a hint for the fixer to suggest some other repairs.'''
                    
    )

register_function(
        rewrite_dsl_file,
        caller = code_fixer,
        executor=executor,
        name="rewrite_dsl_file",
        description='''This function rewrite the dsl file.
        It saves the DSL code generated by the code_fixer.'''
    )

In [27]:
user_proxy.register_nested_chats(
    trigger=code_fixer,
    chat_queue=[
        {
            "sender": executor,
            "recipient": user_proxy,
            "summary_method": "last_msg",
        }
    ],
)

code_fixer.register_nested_chats(
    trigger=user_proxy,
    chat_queue=[
        {
            "sender": executor,
            "recipient": code_fixer,
            "summary_method": "last_msg"
        }
    ]
)

In [28]:
stdout_buffer = io.StringIO()
stderr_buffer = io.StringIO()

with contextlib.redirect_stdout(stdout_buffer), contextlib.redirect_stderr(stderr_buffer):
    try:
        code = f.load(dsl_path)
        f.solve(partial_model,"1",f"{domain}.conforms")
        f.extract("0","0","0")
    except Exception as e:
        pass

message = stdout_buffer.getvalue()
error_message = stderr_buffer.getvalue()

print("Captured stdout:", message)
print("Captured stderr:", error_message)

Captured stdout: (Compiled) MappingExample.4ml
0.04s.
Parsing text took: 0
Visiting text took: 0
Started solve task with Id 1.
0.05s.
Model not solvable. Unsat core terms below.
Conflicts: Mapping.badMapping 
Conflicts: Mapping.invalidUtilization 

0.00s.

Captured stderr: 


In [29]:
user_proxy.initiate_chat(
    recipient=code_fixer,
    message=f'''Here is the DSL domain-model pair loaded: 
    {code} 
    
    The partial model is unsolvable. 
    Please provide some suggestions to modify the constraints of the domain to make the model solvable.
    
    Here is the messages that you can use as a hint to fix the constraints: 
    {message}''',
    max_turns=2
)

User (to Code Fixer):

Here is the DSL domain-model pair loaded: 
    domain Mapping
{
  Component ::= new (id: Integer, utilization: Real).
  Processor ::= new (id: Integer).
  Mapping   ::= new (c: Component, p: Processor).

  // The utilization must be > 50
  invalidUtilization :- c is Component, c.utilization <= 50.

  badMapping :- p is Processor, 
		s = sum(0.0, { c.utilization |
			       c is Component, Mapping(c, p) }), s > 100.

  conforms no badMapping, no invalidUtilization.
}

partial model pm of Mapping
{
  c1 is Component(0, x).
  c2 is Component(1, y).
  p1 is Processor(0).
  Mapping(c1, p1).
  Mapping(c2, p1).
} 
    
    The partial model is unsolvable. 
    Please provide some suggestions to modify the constraints of the domain to make the model solvable.
    
    Here is the messages that you can use as a hint to fix the constraints: 
    (Compiled) MappingExample.4ml
0.04s.
Parsing text took: 0
Visiting text took: 0
Started solve task with Id 1.
0.05s.
Model not so

InternalServerError: Error code: 500 - {'error': {'message': 'Invalid control character at: line 1 column 66 (char 65)', 'type': None, 'param': None, 'code': 500}}